In [35]:
import json
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import json
from pprint import pprint
import os

In [50]:
def parse_json_for_burndown(query_id):
    current_path = os.path.dirname(os.path.realpath("__file__"))
    file = open(os.path.join(current_path,'../../../AustinBot/mockData.json'), 'r')
    mock = json.load(file)
    sprints = mock["sprint"]
    for sprint in sprints:
        if sprint["id"]==query_id:
            current_sprint = sprint
    total_hours = 0
    for story in current_sprint["stories"]:
        total_hours += story["story_hours"]
    #print(current_sprint)
    work_done_on = dict()
    for story in current_sprint["stories"]:
        for task in story["task"]:
            for work in task["daily_progress"]:
                try:
                    work_done_on[work["date"]] += work["work_done"]
                except:
                    work_done_on[work["date"]] = work["work_done"]
    #print(work_done_on)
    #print(total_hours)
    #print(current_sprint)

    x = list(work_done_on.keys())
    y = []
    hours_left = total_hours
    for date in work_done_on:
        hours_left -= work_done_on[date]
        y.append(hours_left)
    print(x)
    print(y)
    each_day = total_hours/len(x)
    hours_left = total_hours
    y_ideal = []
    for i in x:
        hours_left -= each_day
        y_ideal.append(hours_left)
    return [x,y, y_ideal]

In [51]:
def plot_burndown(x, y, y_ideal):
    plotly.tools.set_credentials_file(username='udeshmu', api_key='qIyD3uwDHJdtNvjSsFyS')
    trace1 = go.Scatter(
        x = x,
        y = y,
        name = '<b>Burndown</b>', # Style name/legend entry with html tags
        connectgaps=True
    )
    trace2 = go.Scatter(
        x = x,
        y = y_ideal,
        name = '<b>Ideal</b>', # Style name/legend entry with html tags
        connectgaps=True,
        line = dict(color = ('rgb(205, 12, 24)'),width = 4)
    )
    data = [trace1, trace2]
    fig = dict(data=data)
    #py.image.save_as(fig, filename=os.path.join(current_path,'../Plots/burndown_performance.png'))
    return fig

In [53]:
[x,y,y_ideal] = parse_json_for_burndown("21")
fig=plot_burndown(x,y,y_ideal)

['07/24/2017', '07/25/2017', '07/26/2017', '07/27/2017', '07/28/2017', '07/31/2017', '08/1/2017', '08/2/2017', '08/3/2017', '08/4/2017', '08/7/2017', '08/8/2017', '08/9/2017', '08/10/2017', '08/11/2017']
[455, 431, 407, 385, 360, 335, 308, 277, 250, 223, 195, 166, 139, 115, 91]


In [54]:
py.iplot(fig, filename = 'burndown')